# Viterbi algorithm

This algorithm is useful to solve a problem that is different from what the forward and barckward algorithms solve.

The Viterbi algorithm solves the following problem: given an observed sequence $x_1,...x_T$, what is the most probable sequence of hidden states $z_1,...,z_T$ that is associated with it?

The Viterbi algorithm is an example of dynamic programming.

Even though the Viterbi algorithm is not the forward algorithm, it works somehow *similar* to it, but with a different quantity.

So, what we are trying to find is $$z^* \equiv \text{argmax}_{z}\ p(z| x)\,, $$ where, here, $z$ and $x$ represent, loosely, the entire sequences of hidden and observed states, respectively. Note that $ p(z|x) = \frac{p(z,x)}{p(x)}$ and hence, since $p(x)$ does not depend on $z$, we have that $$ z^* = \text{argmax}_{z}\ p(z,x)$$

In [1]:
import numpy as np

### Solve the problem by brute force

Before going through the Viterbi algorithm, we will see how to solve the problem by brute force, that is, evaluating all the possible combinations or paths in the case of sequence length equal to one and two.

#### 1. Case $T=1$

This is very simple. Assume for example that we have a coin, so that $x \in \{\text{head}, \text{tail}\}$. Also, we assume that the are two possible hidden states, $z \in \{1, 2\}$.

Consider that $\pi = [0.5,0.5]$ and $B(1, \text{head}) = 0.9\,, B(2, \text{head}) = 0.1$. We only need these two quantities because we are going to assume that the observation is $x = \text{head}$ (although note that the above implies $B(1, \text{tail}) = 0.1$ and $B(2, \text{tail}) = 0.9$). It is clear that

$$ p(1,\text{head}) = \pi(1) B(1,\text{head}) = 0.5\times 0.9 = 0.45\,,\\
p(2,\text{head}) = \pi(2) B(2,\text{head}) = 0.5\times 0.1 = 0.05\,,$$

so we conclude that, in this example, the most probable hidden state associated to observation $x = \text{head}$ in a sequence of length $T=1$ is $z = 1$. Note that this result depends on $\pi$ and $B(z,x=\text{head})$

#### 2. Case $T=2$

Now assume that we have two elements in the sequence and that the transition matrix $A_{ij} = p(z_{t+1} = j| z_t = i)$ is 

$$ A= \begin{pmatrix}
0.8 & 0.2\\
0.4 & 0.6 
\end{pmatrix}\,,$$

se that $p(2|1) = 0.2$, for example. The values of $\pi$ and $B$ are the same as before.

Also, assume that the observed sequence is $x = \{\text{heads}, \text{tails}\}$. Now, we know that the brute-force procedure grows exponentially, but $T=2$ is still small enough to handle the computation explicitly.

The set of possible hidden-state sequences is $\{11,12,21,22\}$:

$z=\{11\}$: $\pi(1)B(1, \text{heads}) A(1,1)B(1,\text{tails}) = 0.5\times 0.9 \times 0.8 \times 0.1 = 0.036$

$z=\{12\}$: $\pi(1)B(1, \text{heads}) A(1,2)B(2,\text{tails}) = 0.5 \times 0.9 \times 0.2 \times 0.9 = 0.081$

$z=\{21\}$: $\pi(2)B(2, \text{heads}) A(2,1)B(1,\text{tails}) = 0.5 \times 0.1 \times 0.4 \times 0.1 = 0.002$

$z=\{22\}$: $\pi(2)B(2, \text{heads}) A(2,2)B(2,\text{tails}) = 0.5 \times 0.1 \times 0.6 \times 0.9 = 0.027$

We see that, for this set of parameters, the maximum probability corresponds to $z=\{12\}$ or $z_1 =1\,, z_2=2$.

As for the forward algorithm, we see some commonalities here: the first two elements ($\pi(1)B(1, \text{heads})$) are the same for the  first two combinations, and the same happens ($\pi(2)B(2, \text{heads})$) for the last two.

This can be used to formulate a recursive algorithm whose computation time is only polynomial in $T$.

## Viterbi algorithm

For the recursive algorithm to work we have to define the following quantity:

    delta(t,j): it is the maximum probability, at time t, to arrive at state j given that we have already visited the optimal sequence of states prior to time t.

That is, if I assume that I have passed through the path $z_1,...z_{t-1}$ that has the maximum probability given the observations $x_1, ...x_{t-1}$, then $\delta(t,j)$ is the maximum probability to arrive in state $j$. Maximum in which sense? In the sense that I can reach  $j$ at time $t$ from all the previous states at $t-1$. That is, the recursive formula reads 

$$ \delta(t, j) = \text{max}_{i\in \{1,...,M\}} \left[\delta(t-1,i) A(i,j)\right] B(j, x_t)\,, \hspace{1cm} j=1,...,M $$

Note that the $\text{max}$ condition only affects the term between brackets, but we also mutiply by $B(j, x_t)$ to regard $\delta$ as a joint probability. Also, in order to compute $\delta(t,j)$ we need the value of delta for all possible states associated to $z_{t-1}$ (all the $i$ s). That is, at each timestep $t$ we need to compute the values of $\delta$ for all the states $1,..., M$.

Actually, the above formula is the **induction step**. The **initialization step** is as follows $$ \delta(t=1, j) = \pi(j) B(j,x_1)$$

Finally, the **termination step**, which gives us the maximum probability (associated to the optimal sequence of $z$ states) is $$p^* = \text{max}_{i\in \{1,...,M\}} \delta (t=T, j) $$

As stated above, this is similar to the forward algorithm, but with a $\text{max}$ instead of a $\text{sum}$ function.

Now, there is something weird with the above. How come a recursive algorithm exists? How can we be positive that maximizing the probability *locally* at each timestep we will end up with the *global* maximum probability?

To answer this question, consider the case $T=3$. Supposse that the observation sequence is $x =\{\text{heads},\text{tails},\text{heads}\}$ (same as example with $T=2$ albeit with an extra $\text{heads}$ at the end). We already computed above that the maximum probability for the firsts two steps is attached to $z=\{12\}$ or $z_1 =1\,, z_2=2$ and we know that $p(1,\text{head}) > p(2,\text{head})$, so we only need to make use of the following $\delta$ s at $t=2$:

$$
\delta(t=2, 1) = \pi(1)B(1, \text{heads}) A(1,1)B(1,\text{tails}) = 0.036\\ 
\delta(t=2, 2) = \pi(1)B(1, \text{heads}) A(1,2)B(2,\text{tails}) = 0.081\,.
$$

We now compute $\delta(t=3, 1)$ and $\delta(t=3, 2)$ using the recursive formula:

$$
\delta(t=3, 1) = \text{max}_{i\in \{1,...,M\}} \delta(t=2, i)A(i,1) B(1,\text{heads}) = \delta(t=2, 2)A(2,1)B(1,\text{heads})=0.02916\\ 
\delta(t=3, 2) = \text{max}_{i\in \{1,...,M\}} \delta(t=2, i)A(i,2) B(2,\text{heads}) = \delta(t=2, 2)A(2,2)B(2,\text{heads})=0.00486
$$

So the termination step implies $$p^* = \text{max}_{i\in \{1,...,M\}} \delta (t=T, j) =  \delta (3, 1) = 0.02916$$

So what if we now want to determine the optimal *sequence* $z_1, ...,z_T$. We do have the optimal probability $p^*$, but not the sequence of states that it is related to. To get que optimal sequence of hidden states, we work backwards: we know that $z^*_3 = 1$. Also, note that $\delta(t=3, 1)$ makes use of $\delta(t=2, 2)$ (see above), so $z^*_2 = 2$. Finally, $\delta(t=2, 2)$ makes use of $\delta(t=1, 1) = \pi(1)B(1,\text{heads})$. So we conclude that the optimal sequence is $$z^* = \{1,2,1\}\,,$$ recovering also the first two steps of the optimal sequence $\{1,2\}$ that we computed before. 

In any case, the reason why a greedy algorithm such as Viterbi leads to the global probability is because the transitions are Markov. It does not matter how we arrived at state $i$ at $t-1$ in order to compute the probabilities to arrive at state $j$ at $t$ ("assuming the previous state at $t-1$ is $2$, what is the probability of state $1$ at $t$?" The answer only depends on the fact that we are at $2$, and not on how we arrived at $2$). This implies that we can optimize *locally*, namely, for each timestep, we compute the maximum probability to arrive at each one of the possible states from the **previous** state. If the problem were not Markov and $p(z_t)$ was dependant on $z_{t-1}$ and also $z_{t-2}$, then we would have to extend the Viterbi algorithm to compute optimal probabilities from the two previous states. In any case, a recursive algorthm would still be possible.

Finally, let us mention that, the trick that we used before to compute the optimal sequence is a general technique called *backtracking*, common to dynamic programming algorithms. So once we know the optimal state at $T$, we work backwards to discover the previous optimal states as the ones used in the $\delta$ s. To formalize this intuition, we define 

$$\Psi(t,j) = \text{argmax}_{i\in \{1,...,M\}} \delta(t-1,i) A(i,j)$$

which returns the optimal previous state $i$ that led us to state $j$ at time $t$. The **initialization step** does not matter much and we define it as 

$$\Psi(t=1, j) = 0$$

The termination step gives us the optimal state (initial state of the backtracking algorithm): $$ z^*_T = \text{argmax}_{i\in \{1,...,M\}} \delta(t=T,i)$$

The backtracking algorithm runs backwards to keep obtaining $z^*_{t-1}$ given $z^*_{t}$ (and hence starting from $z^*_{T}$): 

$$z^*_{t-1} = \Psi(t,z^*_{t}) $$

A tensorized version of the Viterbi algorithm for fixed input sequence can be found in notebook *unsupervised_version.ipynb*